In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

In [ ]:
from tqdm import tqdm
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

hand_signs = os.listdir("data/test")

images = []
labels = []

for sign in hand_signs:
    path = "data/test/" + sign
    
    for image in tqdm(os.listdir(path), desc=f"Loading images for {sign}"):
        img = cv.imread(path + "/" + image)
        img = cv.resize(img, (380, 380))
        if img is not None:
            images.append(img)
            labels.append(sign)
    
print("Converting training images to np array with float32 values scaled between 0.0 to 1.0")
X = np.array(images).astype("float32") / 255.0

print("Encoding Y properly to a understandable format")
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

print("One Hot Encoding training labels")
y = y.reshape(-1, 1)
one_hot_encoder = OneHotEncoder()
Y = one_hot_encoder.fit_transform(y)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense

def create_vgg_model(input_shape=(380,380,3), num_classes=13):
    with tf.device("/gpu:0"):
        base_model = VGG16(
            include_top=False,
            input_shape=input_shape,
            pooling="avg",
            weights="imagenet"
        )
        
        # Freeze base model layers
        for layer in base_model.layers[:-4]:
            layer.trainable = False
        
        # Add custom classification head
        x = Flatten()(base_model.output)
        x = Dense(1024, activation="relu", name="fc1")(x)
        x = Dense(512, activation="relu", name="fc2")(x)
        output = Dense(num_classes, activation="softmax", name="predictions")(x)
        
        model = Model(inputs=base_model.input, outputs=output, name='VGG_Model')
        
        return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.sparse import issparse

def train_model(model, X, Y):
    if issparse(X):
        X = X.toarray()
    if issparse(Y):
        Y = Y.toarray() 
    
    # Ensure X and Y are numpy arrays
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        metrics=['accuracy']
    )
    
    batch_size = 8
    
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=3, 
            min_lr=1e-6
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=5, 
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            './models/best_VGG_Classifier.keras', 
            save_best_only=True, 
            monitor='val_accuracy'
        )
    ]
    
    validation_split = 0.2
    split_index = int(len(X) * (1 - validation_split))
    X_train, X_val = X[:split_index], X[split_index:]
    Y_train, Y_val = Y[:split_index], Y[split_index:]
    
    # Use datagen.flow for batch generation from arrays
    train_gen = datagen.flow(X_train, Y_train, batch_size=batch_size)
    val_gen = datagen.flow(X_val, Y_val, batch_size=batch_size)
    
    # Training the model
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        steps_per_epoch=len(X_train) // batch_size,
        validation_steps=len(X_val) // batch_size,
        epochs=20,
        callbacks=callbacks,
    )
    
    return history


In [ ]:
model = create_vgg_model(num_classes=len(hand_signs))

model.summary()

history = train_model(model, X, Y)

model.save("./models/final_VGG_Classifier.keras")